# BT04: Ẩn tin mật trên âm thanh bằng phương pháp echo


Họ tên: Trịnh Văn Minh

MSSV: 1712601

---
## Cách làm bài và nộp bài
&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.


*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `BT04`
        - File `BT04-AnTinMatTrenAmThanh_Echo.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác cấu trúc này.</font>

---
## Import

In [1]:
import numpy as np
from bitarray import bitarray
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio

# YOUR CODE HERE (OPTION)

---
## Các hàm hỗ trợ

In [ ]:
# YOUR CODE HERE (OPTION)

---
## Hàm nhúng (4đ)

Trong bài này, để đơn giản, ta sẽ chỉ làm với file âm thanh mono (một sóng) và không mất mát thông tin (*.wav).

Với cách rút trích đơn giản được cài đặt trong bài tập này, sẽ có một số bit bị rút trích sai và ta muốn biết tỉ lệ % các bit bị rút trích sai là bao nhiêu. Để làm điều này, ta sẽ tách phần tính message bits (đã bao gồm 100...) từ message file ra khỏi hàm nhúng; phần này đã được viết sẵn cho bạn trong hàm `compute_msg_bits` ở bên dưới. Lúc này, input của hàm nhúng là message bits (đã bao gồm 100...), output của hàm rút trích là message bits được rút trích (vẫn chứa 100...), và ta có thể so sánh 2 message bits và tính tỉ lệ % các bit bị sai.

Một số lưu ý để bạn ra được kết quả giống với của mình:
- Khi chuyển từ độ trễ theo đơn vị giây sang độ trễ theo đơn vị sample, bạn ép kiểu `int` (làm tròn xuống).
- Khi tính số lượng sample trên mỗi đoạn (mỗi đoạn sẽ nhúng một bit), nếu ra lẻ thì cũng ép kiểu `int` (làm tròn xuống); như vậy, trong stego audio, ở đoạn cuối (đoạn bị lẻ) sẽ không có echo.

In [32]:
def compute_msg_bits(msg_file, cover_aud_file, num_segments):
    '''
    Tính message bits (đã có đuôi 100...) từ message file.
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_aud_file (str): Tên file chứa cover audio.
        num_segments (int): Số lượng đoạn sóng dùng để nhúng (mỗi đoạn sẽ nhúng một bit).
    Giá trị trả về:
        Mảng numpy (một chiều, dtype = bool): message bits (đã thêm 100...), 
                                              chiều dài của mảng này = num_segments,
                                              nếu không đủ chỗ nhúng thì trả về mảng rỗng.
    '''
    # Đọc msg file
    with open(msg_file, 'r') as f:
        msg = f.read()
    
    # Chuyển msg thành msg bits
    msg_bits = bitarray()
    msg_bits.frombytes(msg.encode('utf-8'))
   
    # Kiểm xem có nhúng được không?
    _, cover_samples = wavfile.read(cover_aud_file)
    if len(msg_bits) + 1 > num_segments:
        print('Cannot embed!')
        return np.array([])  
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (num_segments - len(msg_bits) - 1))
    return np.array(list(msg_bits))

In [132]:
def embed(msg_bits, cover_aud_file, num_segments, time_delta0, time_delta1, decay_rate, 
          stego_aud_file):
    '''
    Nhúng tin mật trên âm thanh bằng phương pháp echo.
    
    Các tham số:
        msg_bits (mảng numpy): Mảng (một chiều, dtype=bool) ứng với message bits (đã thêm 100...),
                               len(msg_bits) = num_segments.
        cover_aud_file (str): Tên file chứa cover audio.
        num_segments (int): Số lượng đoạn sóng dùng để nhúng (mỗi đoạn sẽ nhúng một bit).
        time_delta0 (float): Độ trễ (giây) của echo ứng với bit 0.
        time_delta1 (float): Độ trễ (giây) của echo ứng với bit 1.
        decay_rate (float): Giá trị từ 0-1, cho biết mức độ giảm cường độ của echo so với sóng ban đầu
                            (echo = decay_rate * sóng ban đầu dịch phải).
        stego_aud_file (str): Tên file chứa stego audio (kết quả sau khi nhúng).       
    '''
    # YOUR CODE HERE
    if msg_bits.size == 0:
        return False

    rate, samples = wavfile.read(cover_aud_file)

    sample_delay0 = int(time_delta0 * rate)
    sample_delay1 = int(time_delta1 * rate)

    echo0 = np.zeros_like(samples)
    echo1 = np.zeros_like(samples)

    echo0[sample_delay0:] = samples[:-sample_delay0] * decay_rate
    echo1[sample_delay1:] = samples[:-sample_delay1] * decay_rate

    samples_echo0 = samples + echo0
    samples_echo1 = samples + echo1

    #cắt thành 64 phần
    mixer1 = msg_bits
    mixer0 = 1 - msg_bits


    #final step
    stego_samples = np.empty_like(samples)

    b = 0 #chỉ số duyệt msg_bits
    part_vol = len(samples)//num_segments

    for i in range(0, len(samples), part_vol):
        if b < num_segments:
            if msg_bits[b] == True:
                stego_samples[i: i + part_vol] = samples_echo1[i: i + part_vol]
            elif msg_bits[b] == False:
                stego_samples[i: i + part_vol] = samples_echo0[i: i + part_vol]
            b += 1
        else:
            stego_samples[i: i + part_vol] = samples[i: i + part_vol]
    
 
    # Ghi stego audio xuống file
    wavfile.write(stego_aud_file, rate, stego_samples)
    
    return True

In [145]:
# TEST
num_segments = 64; time_delta0 = 0.03; time_delta1 = 0.031; decay_rate = 0.1
msg_bits = compute_msg_bits('msg.txt', 'cover.wav', num_segments)
embed(msg_bits, 'cover.wav', num_segments, time_delta0, time_delta1, decay_rate, f'stego_{decay_rate}.wav')
_, correct_cover_samples = wavfile.read('correct_stego.wav')
_, cover_samples = wavfile.read('stego.wav')
assert np.array_equal(correct_cover_samples, cover_samples)

In [134]:
# Nghe thử cover audio
Audio(filename='cover.wav')

In [ ]:
# Nghe thử stego audio
Audio(filename='stego.wav')

---
## Hàm rút trích (4đ)

Mình mới cập nhật file "09-Demo.ipynb", mục "Demo tạo sóng có echo và tìm echo bằng autocorrelation"; trong đó nói rõ hơn về autocorrelation và cách rút trích bit bằng autocorrelation. Trước khi cài đặt hàm rút trích ở bên dưới, bạn lấy lại file "09-Demo.ipynb" (ở đầu file mới có dòng "Cập nhật ngày 30/01/2020") và đọc mục "Demo tạo sóng có echo và tìm echo bằng autocorrelation" để hiểu cách rút trích.

Luật rút trích như sau: với mỗi đoạn sóng:
- Nếu autocorrelation tại độ trễ của echo ứng với bit 0 $\color{blue}>$ autocorrelation tại độ trễ của echo ứng với bit 1: bit = 0
- Ngược lại: bit = 1

In [141]:
def extract(stego_aud_file, num_segments, time_delta0, time_delta1):
    '''
    Rút trích message bits đã được nhúng trên âm thanh bằng phương pháp echo.
    
    Các tham số:
        stego_aud_file (str): Tên file chứa stego audio.
        num_segments (int): Số lượng đoạn sóng dùng để nhúng (mỗi đoạn sẽ nhúng một bit).
        time_delta0 (float): Độ trễ (giây) của echo ứng với bit 0.
        time_delta1 (float): Độ trễ (giây) của echo ứng với bit 1.
    
    Giá trị trả về:
        Mảng numpy (một chiều, dtype = bool): Message bits được rút trích (gồm cả đuôi 100...).
    '''
    # YOUR CODE HERE
    rate, samples_echo = wavfile.read(stego_aud_file)

    #k: độ lệch
    k0 = int(time_delta0 * rate)
    k1 = int(time_delta1 * rate)

    extr_msg_bits = bitarray()
    part_vol = len(samples_echo)//num_segments
    for i in range(0, len(samples_echo) - part_vol + 1, part_vol):
        part = samples_echo[i: i + part_vol]
        temp = (part - part.mean()) / (part.var() ** 0.5) 
        if k0 == 0 or k1 == 0: 
            ac = np.mean(temp * temp)
        else:
            ac0 = np.mean(temp[:-k0] * temp[k0:])
            ac1 = np.mean(temp[:-k1] * temp[k1:])
            extr_msg_bits.extend([ac0 < ac1])

    # # Cắt đuôi '100...' ra khỏi msg bits
    # extr_msg_bits = extr_msg_bits[:extr_msg_bits.to01().rfind('1')]
    
    # # Ghi msg xuống file
    # with open('extr_msg.txt', 'wb') as f:
    #     f.write(extr_msg_bits)

    return np.array(list(extr_msg_bits))

In [142]:
# TEST
num_segments = 64; time_delta0 = 0.03; time_delta1 = 0.031
extr_msg_bits = extract('correct_stego.wav', num_segments, time_delta0, time_delta1)
err = np.mean(msg_bits != extr_msg_bits) * 100 # Tỉ lệ % các bit bị rút trích sai
assert round(err, 3) == 1.562

---
## Ảnh hưởng của `decay_rate` (2đ)

Để thấy được ảnh hưởng của `decay_rate`, ta sẽ thử nhúng và rút trích với 3 giá trị `decay_rate`: 1, 0.5, 0.1 (các tham số `num_segments`, `time_delta0`, `time_delta1` vẫn giữ nguyên như ở trên).

In [146]:
num_segments = 64; time_delta0 = 0.03; time_delta1 = 0.031
for decay_rate in [1.0, 0.5, 0.1]:
    stego_aud_file = f'stego_{decay_rate}.wav'
    msg_bits = compute_msg_bits('msg.txt', 'cover.wav', num_segments)
    embed(msg_bits, 'cover.wav', num_segments, time_delta0, time_delta1, decay_rate, stego_aud_file)
    extr_msg_bits = extract(stego_aud_file, num_segments, time_delta0, time_delta1)
    err = np.mean(msg_bits != extr_msg_bits) * 100
    print(f'decay_rate: {decay_rate}, error (%): {err}')

decay_rate: 1.0, error (%): 1.5625
decay_rate: 0.5, error (%): 4.6875
decay_rate: 0.1, error (%): 35.9375


In [147]:
Audio(filename='stego_1.0.wav') # decay_rate = 1.0

In [ ]:
Audio(filename='stego_0.5.wav') # decay_rate = 0.5

In [ ]:
Audio(filename='stego_0.1.wav') # decay_rate = 0.1

Từ kết quả trên, cho biết khi `decay_rate` giảm thì tính vô hình và độ chính xác rút trích thay đổi như thế nào? Bạn thử nghĩ xem tại sao kết quả lại như vậy (có sao thì nói vậy, chưa nghĩ ra thì nói là chưa nghĩ ra)?

YOUR ANSWER HERE